In [1]:
import json
import urllib.request
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
QUERY_URL = "https://www.alphavantage.co/query?function={REQUEST_TYPE}&apikey={KEY}&symbol={SYMBOL}"
API_KEY = 'RG3MY4ZIWNEBVF8F'

def _request(symbol, req_type):
    with urllib.request.urlopen(QUERY_URL.format(REQUEST_TYPE=req_type, KEY=API_KEY, SYMBOL=symbol)) as req:
        data = req.read().decode("UTF-8")
    return data

def get_monthly_data(symbol):
    return json.loads(_request(symbol, 'TIME_SERIES_MONTHLY'))

def get_daily_data(symbol):
    return json.loads(_request(symbol, 'TIME_SERIES_DAILY'))

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
company_df = pd.read_excel('company_for_priceindex.xls')

#### Create the first one

In [5]:
company_list =  company_df.company_index.tolist()

company_data_monthly = pd.DataFrame(columns = ['yyyy-mm-dd'])
for company in company_list:
    df = pd.DataFrame.from_dict(json_normalize(get_monthly_data(company)))
    cols_to_use = [i for i in df.columns if 'Monthly' in i and 'close' in i] #data at close
    cols_to_use_renamed = {i:i.replace('Monthly Time Series.', '')[:7] for i in cols_to_use}

    df = df.rename(columns = cols_to_use_renamed)[cols_to_use_renamed.values()].T
    data = pd.DataFrame(columns = ['yyyy-mm-dd', company])
    data[company] = df[0].tolist()
    data['yyyy-mm-dd'] = df.index.tolist()
#
    company_data_monthly = company_data_monthly.merge(data, how = 'outer', on = 'yyyy-mm-dd')
    
comp_notnull = company_data_monthly.loc[:,company_data_monthly.isnull().sum() != company_data_monthly.shape[0]].columns.tolist()
company_list_remaining = [i for i in company_list if i not in comp_notnull]

In [6]:
company_data_monthly

,yyyy-mm-dd,WMT,CLX,JNJ,PG,ROKU,NFLX,GRUB,AMZN,CPB,...,HLT,MAR,EXPE,CVS,ANTM,HUM,UNH,DFS,BAC,AXP
0,2020-03,113.9700,177.4000,119.8900,102.4300,76.1300,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02,107.6800,159.4200,134.4800,113.2300,113.6700,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01,114.4900,157.3100,148.8700,124.6200,120.9500,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-12,118.8400,153.5400,145.8700,124.9000,133.9000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-11,119.0900,148.2300,137.4900,122.0600,160.3700,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2000-08,47.6300,36.1900,91.9600,61.8300,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,2000-07,55.2500,41.3100,93.0600,57.0000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,2000-06,57.6300,44.8100,101.9000,56.7500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,2000-05,57.6300,39.6200,89.5000,66.5000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
company_data_monthly = company_data_monthly.loc[:,comp_notnull]

#### Repeat a few times

In [56]:
for company in company_list_remaining:
    df = pd.DataFrame.from_dict(json_normalize(get_monthly_data(company)))
    cols_to_use = [i for i in df.columns if 'Monthly' in i and 'close' in i] #data at close
    cols_to_use_renamed = {i:i.replace('Monthly Time Series.', '')[:7] for i in cols_to_use}

    df = df.rename(columns = cols_to_use_renamed)[cols_to_use_renamed.values()].T
    data = pd.DataFrame(columns = ['yyyy-mm-dd', company])
    data[company] = df[0].tolist()
    data['yyyy-mm-dd'] = df.index.tolist()
#
    company_data_monthly = company_data_monthly.merge(data, how = 'outer', on = 'yyyy-mm-dd')
    
comp_notnull = company_data_monthly.loc[:,company_data_monthly.isnull().sum() != company_data_monthly.shape[0]].columns.tolist()
company_list_remaining = [i for i in company_list if i not in comp_notnull]

In [57]:
company_data_monthly = company_data_monthly.loc[:,comp_notnull]

In [58]:
company_data_monthly.head()

,yyyy-mm-dd,WMT,CLX,JNJ,PG,ROKU,NFLX,GRUB,AMZN,CPB,...,KR,AXP,COST,BUD,USO,DJI,SBUX,MCD,DAL,UAL
0,2020-03,113.9700,177.4000,119.8900,102.4300,76.1300,332.8300,31.7100,1846.0900,45.3000,...,31.7700,74.1200,290.4200,40.3000,4.9400,19173.9805,58.0300,148.4900,21.3500,24.5000
1,2020-02,107.6800,159.4200,134.4800,113.2300,113.6700,369.0300,48.1100,1883.7500,45.1200,...,28.1300,109.9300,281.1400,58.3700,9.4500,25409.3594,78.4300,194.1700,46.1300,61.5900
2,2020-01,114.4900,157.3100,148.8700,124.6200,120.9500,345.0900,54.1500,2008.7200,48.3900,...,26.8600,129.8700,305.5200,75.3000,10.8400,28256.0293,84.8300,213.9700,55.7400,74.8000
3,2019-12,118.8400,153.5400,145.8700,124.9000,133.9000,323.5700,48.6400,1847.8400,49.4200,...,28.9900,124.4900,293.9200,82.0400,12.8100,28538.4395,87.9200,197.6100,58.4800,88.0900
4,2019-11,119.0900,148.2300,137.4900,122.0600,160.3700,314.6600,43.1200,1800.8000,46.5700,...,27.3400,120.1200,299.8100,79.3500,11.6200,28051.4102,85.4300,194.4800,57.3100,92.8000


In [6]:
company_data_monthly

,yyyy-mm-dd,WMT,CLX,JNJ,PG,ROKU,CHTR,VZ,PTON,UBER,...,DAL,UAL,PLNT,EQX,CCL,HLT,MAR,EXPE,CVS,ANTM
0,2020-03-13,114.10,167.77,134.29,114.07,80.95,454.19,54.17,19.72,22.60,...,38.36,41.64,53.98,5.75,17.58,78.63,95.61,69.63,59.64,270.34
1,2020-03-12,104.05,158.70,125.41,101.84,78.81,427.54,51.20,19.51,22.61,...,33.71,37.08,48.32,5.86,14.97,76.72,93.71,65.04,54.60,256.21
2,2020-03-11,114.43,169.40,131.80,111.59,90.63,450.65,54.83,22.00,26.24,...,42.67,49.34,57.00,6.62,21.75,81.09,100.11,76.67,61.31,281.69
3,2020-03-10,119.79,175.00,141.64,120.55,97.30,476.76,56.52,22.97,28.97,...,45.47,52.56,62.63,7.70,24.02,90.32,109.98,85.93,62.60,286.05
4,2020-03-09,117.16,174.90,136.44,116.06,96.26,463.21,55.83,23.21,28.17,...,43.52,46.78,56.65,7.72,21.74,86.18,104.19,86.30,61.30,269.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2019-10-25,119.04,151.40,128.35,123.25,133.74,462.19,60.37,22.40,32.71,...,54.60,91.64,61.04,6.23,44.57,97.69,126.18,137.06,65.10,263.07
96,2019-10-24,119.10,150.76,127.50,124.79,131.22,435.07,60.58,21.34,33.28,...,53.79,89.81,61.11,6.11,43.90,96.91,124.30,137.14,64.84,259.49
97,2019-10-23,119.35,149.81,129.90,123.00,128.76,435.56,60.88,21.08,33.05,...,53.81,89.90,60.12,6.06,44.00,95.77,123.49,138.31,65.60,263.51
98,2019-10-22,119.58,148.44,129.20,122.18,129.06,437.23,60.77,21.16,32.53,...,54.65,89.71,60.46,6.05,42.55,90.73,122.05,136.65,66.56,260.25
